In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [2]:
r1 = requests.get('https://www.elelmiszerbank.hu/hu/hirek/hireink.html')
coverpage = r1.content
soup1 = BeautifulSoup(coverpage, 'html5lib')

In [3]:
coverpage_news = soup1.find_all('div', class_="col-xs-12 col-sm-6 newsSub")

In [4]:
links = []
dates = []
for i in range(len(coverpage_news)):
    date = coverpage_news[i].find('b').get_text()
    link = 'https://www.elelmiszerbank.hu/'+str(coverpage_news[i].find('a').get('href',''))
    if date.startswith('2019') or date.startswith('2020') or date.startswith('2021'):
        dates.append(date)
        links.append(link)

In [5]:
cikkek_adatai = []
start = time.time()
counter = 0
for m in links:
    # link
    article = requests.get(m)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')

    #date
    date = dates[counter]
    
    # cím
    title_raw = soup_article.select("h2")
    if len(title_raw)>0:
        title = title_raw[0].get_text(strip=True)

    # törzsszöveg
    body = soup_article.find('div', class_="col-xs-12 NewsSub")
    szoveg = body.get_text(strip=True)
    
    # putting it together
    cikkek_adatai.append([date, title, szoveg, m])
    end = time.time()
    counter += 1
    print(str(counter) + '/' + str(len(links)), end - start)


1/77 0.22059416770935059
2/77 0.43357229232788086
3/77 0.6139414310455322
4/77 0.797459602355957
5/77 0.9767310619354248
6/77 1.1609561443328857
7/77 1.3412718772888184
8/77 1.5215075016021729
9/77 1.7390880584716797
10/77 1.9037463665008545
11/77 2.079948663711548
12/77 2.268897294998169
13/77 2.452613592147827
14/77 2.637150764465332
15/77 2.8206186294555664
16/77 2.978445291519165
17/77 3.1506571769714355
18/77 3.327958822250366
19/77 3.4947280883789062
20/77 3.6803207397460938
21/77 3.855842351913452
22/77 4.04983377456665
23/77 4.234265327453613
24/77 4.513328313827515
25/77 4.711373567581177
26/77 4.922644853591919
27/77 5.105323791503906
28/77 5.2945380210876465
29/77 5.455806255340576
30/77 5.629277944564819
31/77 5.823836326599121
32/77 6.035254716873169
33/77 6.219106674194336
34/77 6.414527177810669
35/77 6.627329349517822
36/77 6.826884984970093
37/77 6.995508193969727
38/77 7.201028347015381
39/77 7.389436483383179
40/77 7.575804948806763
41/77 7.768033742904663
42/77 7.95

In [6]:
df = pd.DataFrame(data=cikkek_adatai)
df.columns = ['dátum','cím', 'szöveg', 'link']
with pd.ExcelWriter('elemiszerbank.xlsx') as writer:  
    df.to_excel(writer, sheet_name='hu')